Slightly modified version of [this theano tutorial](http://www.marekrei.com/blog/theano-tutorial/).

## Linear regression with Python

## Theano 

Theano is a powerful library dedicated to work with computational graph with many libraries built on top of it. It is also a low level core for our library for probabilistic modeling.

### How to install Theano
To install Theano for Ubuntu 14.04 (or other unix-like machine) use the following code:
```bash
sudo apt-get install python-numpy python-scipy python-dev python-pip python-nose g++ libopenblas-dev git
sudo pip install Theano
```

In [ ]:
import theano
import theano.tensor as T

In [ ]:
# symbolic variable, 32 bit float
x = theano.tensor.fvector('x') # dtype=float32
# shared variable with specified value
W = theano.shared(np.asarray([0.4, 0.7]), 'W')
# another variable
y = (x * W).sum()

# function f takes x as input and produces y as output
f = theano.function([x], y)

# run the function for an example input
output = f([1.0, 2.0])
# 0.4 * 1.0 + 0.7 * 2.0 = 1.8
print(output)

In [ ]:
# Shared variables
W.get_value()

In [ ]:
W.set_value([0.1, 0.8])

In [ ]:
W.get_value()

## Minimal training example with optimization

In [ ]:
x = theano.tensor.fvector('x')
target = theano.tensor.fscalar('target')

In [ ]:
W = theano.shared(np.asarray([0.2, 0.7]), 'W')
y = (x * W).sum()

In [ ]:
cost = theano.tensor.sqr(target - y)
gradients = theano.tensor.grad(cost, [W])
W_updated = W - (0.1 * gradients[0])
updates = [(W, W_updated)]
 

In [ ]:
sample_function = theano.function([x, target], y, updates=updates)

In [ ]:
step_number = 20
w_values = np.zeros((step_number, 2))
output_values = np.zeros((step_number, 1))

for index in range(step_number):
    output = sample_function([1.0, 1.0], 20.0)
    w_values[index, :] = W.get_value()
    output_values[index] = output
    print(W.get_value())
    print(output)

In [ ]:
fig = plt.figure()
plt.plot(w_values[:, 0], w_values[:, 1], '-o')
plt.xlabel('x1')
plt.ylabel('x2')

## Logistic regression with Theano

Sample $D = \{(x_i, y_i)\}_{i = 1}^n$.
Suppose, that $p(y|x, w) = C(x^T w)$.

Denote $t_i =C(x_i^T w)$.
Then the likelihood is the following:
$$
p(D | w) = \prod_{i = 1}^n y_i^{t_i} (1- y_i)^{1 - t_i}.
$$
And the loglikelihood is:
$$
\log p(D | w) = \sum_{i = 1}^n \log(y_i) t_i + \log(1- y_i) * (1 - t_i).
$$
There is no analytical solution, but for most popular $C()$ functions the problem is convex and 
can be solved with common gradient-based numerical optimization approaches.

In [ ]:
random_number_generator = np.random

training_sample_size = 100
input_variable_number = 20

In [ ]:
# Generate a training sample
sample = (random_number_generator.randn(training_sample_size, input_variable_number), 
          random_number_generator.randint(size=training_sample_size, low=0, high=2))
training_steps_number = 500

In [ ]:
# Declare Theano symbolic variables
points = T.dmatrix("points")
values = T.dvector("values")

In [ ]:
# Initialize weights and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)

# Initialize the weight vector randomly
weights = theano.shared(random_number_generator.randn(input_variable_number), name="weights")

In [ ]:
# Initialize the bias term
bias = theano.shared(0., name="bias")

In [ ]:
print("Initial model:")
print(weights.get_value())
print(bias.get_value())

In [ ]:
# Construct Theano expression graph
values_probability = 1 / (1 + T.exp(-T.dot(points, weights) - bias))   # Probability that target = 1

prediction = values_probability > 0.5                    # The prediction thresholded

cross_entropy = -(values * T.log(values_probability) +
                  (1 - values) * T.log(1 - values_probability)) # Cross-entropy loss function

cost = cross_entropy.mean() + 0.01 * (weights ** 2).sum()# The cost to minimize

In [ ]:
# Compute the gradient of the cost
# w.r.t the weight vector and the bias term
gradient_weights, gradient_bias = T.grad(cost, [weights, bias])

In [ ]:
# Compile
train_model = theano.function(
                inputs=[points, values],
                outputs=[prediction, cross_entropy],
                updates=((weights, weights - 0.1 * gradient_weights), 
                         (bias, bias - 0.1 * gradient_bias)))

In [ ]:
predict_values = theano.function(inputs=[points], outputs=prediction)

In [ ]:
training_steps_number

In [ ]:
# Train model
step_error_array = np.zeros((training_steps_number, 1))
for index in range(training_steps_number):
    step_prediction, step_error = train_model(sample[0], sample[1])
    step_error_array[index] = np.sum(step_error)

In [ ]:
# Print results
print("Final model:")
print(weights.get_value())
print(bias.get_value())
print("target values for sample:")
print(sample[1])
print("prediction on sample:")
print(predict_values(sample[0]))

In [ ]:
fig = plt.figure()
plt.plot(step_error_array)
plt.xlabel('Step number')
plt.ylabel('Cross entropy')

In [ ]:
# Homework
# 1. Select regularization parameter using cross validation
# 2. Replace logistic (logit) regression with probit regression
# 3. Compare performace of logit and probit regression for classification datasets available in sklearn:
#    estimate quality and std using cross-validation
